In [25]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.types import IntegerType,DoubleType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix
from pyspark.ml.linalg import Vectors
from pyspark.ml.functions import array_to_vector

In [2]:
spark = pyspark.SparkConf()

In [3]:
spark=SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [4]:
# Read data from CSV file
df = spark.read.csv('C:/Users/uti/Downloads/diabetes.csv',header=True)

In [5]:
df.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [6]:
df.dtypes

[('Pregnancies', 'string'),
 ('Glucose', 'string'),
 ('BloodPressure', 'string'),
 ('SkinThickness', 'string'),
 ('Insulin', 'string'),
 ('BMI', 'string'),
 ('DiabetesPedigreeFunction', 'string'),
 ('Age', 'string'),
 ('Outcome', 'string')]

In [7]:
for x in df.columns:
    if x=="BMI" or x=="DiabetesPedigreeFunction":
        df = df.withColumn(str(x), df[str(x)].cast(DoubleType()))
        continue
    df = df.withColumn(str(x), df[str(x)].cast(IntegerType()))

In [8]:
df.dtypes


[('Pregnancies', 'int'),
 ('Glucose', 'int'),
 ('BloodPressure', 'int'),
 ('SkinThickness', 'int'),
 ('Insulin', 'int'),
 ('BMI', 'double'),
 ('DiabetesPedigreeFunction', 'double'),
 ('Age', 'int'),
 ('Outcome', 'int')]

In [9]:
df.show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [10]:
assembler = VectorAssembler(inputCols=[
    'Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'
], outputCol='features')

# Consolidate predictor columns
data_assembled = assembler.transform(df)

# Check the resulting column
data_assembled=data_assembled.select('features', 'Outcome')


In [11]:
data_assembled.show(10)

+--------------------+-------+
|            features|Outcome|
+--------------------+-------+
|[6.0,148.0,72.0,3...|      1|
|[1.0,85.0,66.0,29...|      0|
|[8.0,183.0,64.0,0...|      1|
|[1.0,89.0,66.0,23...|      0|
|[0.0,137.0,40.0,3...|      1|
|[5.0,116.0,74.0,0...|      0|
|[3.0,78.0,50.0,32...|      1|
|[10.0,115.0,0.0,0...|      0|
|[2.0,197.0,70.0,4...|      1|
|[8.0,125.0,96.0,0...|      1|
+--------------------+-------+
only showing top 10 rows



In [12]:
df_train, df_test = data_assembled.randomSplit([0.8, 0.2], seed=17)
# Check that training set has around 80% of records
training_ratio = df_train.count() / data_assembled.count()

print(training_ratio)

0.796875


In [13]:
df_test.dtypes

[('features', 'vector'), ('Outcome', 'int')]

In [14]:
tree = DecisionTreeClassifier(featuresCol="features", labelCol="Outcome")
tree_model = tree.fit(df_train)

In [15]:
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="Outcome")
dtc = dtc.fit(df_train)

pred = dtc.transform(df_test)
pred.show(3)

+--------------------+-------+-------------+--------------------+----------+
|            features|Outcome|rawPrediction|         probability|prediction|
+--------------------+-------+-------------+--------------------+----------+
|(8,[0,1,6,7],[2.0...|      0|  [108.0,1.0]|[0.99082568807339...|       0.0|
|(8,[1,5,6,7],[73....|      0|  [108.0,1.0]|[0.99082568807339...|       0.0|
|(8,[1,5,6,7],[138...|      1|    [0.0,4.0]|           [0.0,1.0]|       1.0|
+--------------------+-------+-------------+--------------------+----------+
only showing top 3 rows



In [16]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="Outcome")
acc = evaluator.evaluate(pred)
print("Prediction Accuracy: ", acc)

Prediction Accuracy:  0.7730087947479252


In [17]:
y_pred=pred.select("prediction").collect()
y_orig=pred.select("outCome").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[86 15]
 [20 35]]


In [18]:
df_test.toPandas().head(5)


,features,Outcome
0,"(2.0, 74.0, 0.0, 0.0, 0.0, 0.0, 0.102, 22.0)",0
1,"(0.0, 73.0, 0.0, 0.0, 0.0, 21.1, 0.342, 25.0)",0
2,"(0.0, 138.0, 0.0, 0.0, 0.0, 36.3, 0.933, 25.0)",1
3,"[0.0, 84.0, 64.0, 22.0, 66.0, 35.8, 0.545, 21.0]",0
4,"[0.0, 84.0, 82.0, 31.0, 125.0, 38.2, 0.233, 23.0]",0


In [40]:
ourPred=[]
ourPred.append(int(input('Pregnancies: ')))
ourPred.append(int(input('Glucose')))
ourPred.append(int(input('BloodPressure')))
ourPred.append(int(input('SkinThickness')))
ourPred.append(int(input('Insulin')))
ourPred.append(int(input('BMI')))
ourPred.append(int(input('DiabetesPedigreeFunction')))
ourPred.append(int(input('Age')))
# p=tree_model.predict(ourPred)
# print('votre test est: '+ str(p))

Pregnancies: 0
Glucose138
BloodPressure0
SkinThickness0
Insulin0
BMI36
DiabetesPedigreeFunction0
Age25


In [41]:
To_Predict = Vectors.dense(ourPred)
To_Predict
# a=spark.createDataFrame(ourPred, IntegerType())

DenseVector([0.0, 138.0, 0.0, 0.0, 0.0, 36.0, 0.0, 25.0])

In [42]:
prediction = dtc.predict(To_Predict)

In [43]:
prediction

1.0

In [83]:
sc = spark.sparkContext
model_path = "dtc_model"
dtc.write().overwrite().save(model_path)

Py4JJavaError: An error occurred while calling o1525.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.jav